# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
csv_path = r"cities.csv"
cities_df = pd.read_csv(csv_path)
cities_df

,Unnamed: 0,City,Country,Lat,Lng,Temp (F),Humidity,Cloudiness,Wind Speed,Max Temp (F)
0,0,Bluff,NZ,-46.6000,168.3333,48.99,78,96,7.00,48.99
1,1,Busselton,AU,-33.6500,115.3333,63.19,82,1,1.54,63.19
2,2,Porto-Novo,BJ,6.4965,2.6036,84.20,84,40,12.66,84.20
3,3,Tete,MZ,-16.1564,33.5867,73.20,78,0,4.61,73.40
4,4,Barrow,US,71.2906,-156.7887,30.20,80,1,5.75,30.20
...,...,...,...,...,...,...,...,...,...,...
555,555,Moranbah,AU,-22.0016,148.0466,71.47,66,84,20.78,71.47
556,556,Mercedes,AR,-34.6515,-59.4307,60.03,71,2,3.00,62.01
557,557,Clarence Town,AU,-32.5833,151.7833,63.54,63,0,11.50,64.99
558,558,Boa Vista,BR,2.8197,-60.6733,84.20,61,40,6.91,84.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [15]:
# store lat and lon as locations
locations = cities_df[['Lat', 'Lng']].astype(float)

# convert humidity to float
humidity = cities_df['Humidity'].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
# drop NaN values

cities_df = cities_df.dropna()
cities_df

,Unnamed: 0,City,Country,Lat,Lng,Temp (F),Humidity,Cloudiness,Wind Speed,Max Temp (F)
0,0,Bluff,NZ,-46.6000,168.3333,48.99,78,96,7.00,48.99
1,1,Busselton,AU,-33.6500,115.3333,63.19,82,1,1.54,63.19
2,2,Porto-Novo,BJ,6.4965,2.6036,84.20,84,40,12.66,84.20
3,3,Tete,MZ,-16.1564,33.5867,73.20,78,0,4.61,73.40
4,4,Barrow,US,71.2906,-156.7887,30.20,80,1,5.75,30.20
...,...,...,...,...,...,...,...,...,...,...
555,555,Moranbah,AU,-22.0016,148.0466,71.47,66,84,20.78,71.47
556,556,Mercedes,AR,-34.6515,-59.4307,60.03,71,2,3.00,62.01
557,557,Clarence Town,AU,-32.5833,151.7833,63.54,63,0,11.50,64.99
558,558,Boa Vista,BR,2.8197,-60.6733,84.20,61,40,6.91,84.20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
import json
import requests

# Make a new dataframe to store the values
hotel_df = cities_df

#create a blank column named for storing Hotel Name
hotel_df["Hotel Name"]= ""

# Base url for nearbysearch
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up parameters with radius being 5000 meters
params = {
    "radius" : 5000,
    "type": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    params['location']=f'{lat},{lng}'

    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    # print(json.dumps(hotel_name, indent=4, sort_keys=True))
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_name['results'][0]['name']
    except (KeyError, IndexError):
        print(f'Skipping, cannot find hotel within 5000m.')





ipykernel_launcher:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Tracy\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
Skipping, cannot find hotel within 5000m.
Skipping, cannot find hotel within 5000m.
Skipping, cannot find hotel within 5000m.
Skipping, cannot find hotel within 5000m.


In [19]:
#Check dataframe
hotel_df

,Unnamed: 0,City,Country,Lat,Lng,Temp (F),Humidity,Cloudiness,Wind Speed,Max Temp (F),Hotel Name
0,0,Bluff,NZ,-46.6000,168.3333,48.99,78,96,7.00,48.99,Bluff
1,1,Busselton,AU,-33.6500,115.3333,63.19,82,1,1.54,63.19,Busselton
2,2,Porto-Novo,BJ,6.4965,2.6036,84.20,84,40,12.66,84.20,Porto-Novo
3,3,Tete,MZ,-16.1564,33.5867,73.20,78,0,4.61,73.40,Tete
4,4,Barrow,US,71.2906,-156.7887,30.20,80,1,5.75,30.20,Utqiagvik
...,...,...,...,...,...,...,...,...,...,...,...
555,555,Moranbah,AU,-22.0016,148.0466,71.47,66,84,20.78,71.47,Moranbah
556,556,Mercedes,AR,-34.6515,-59.4307,60.03,71,2,3.00,62.01,Mercedes
557,557,Clarence Town,AU,-32.5833,151.7833,63.54,63,0,11.50,64.99,Clarence Town
558,558,Boa Vista,BR,2.8197,-60.6733,84.20,61,40,6.91,84.20,Boa Vista


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map

# using locations as found above
markers = gmaps.marker_layer(locations)

# adding the markers
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))